<a href="https://colab.research.google.com/github/shriman/attraqt_movie_recommendation/blob/main/Attraqt_collaborative_filtering_svd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies here

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install scikit-surprise

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# test runtime config
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


# Imports

In [3]:
import os
import seaborn as sns
from os import path
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval

from surprise import Dataset, SVD, Reader
from surprise.model_selection import cross_validate

QUANTILE = 0.95

data_path = '/content/drive/My Drive/Colab Notebooks/Attraqt'
output_path = '/content/drive/My Drive/Colab Notebooks/Attraqt/outputs'

# Some helper functions

In [4]:
def get_source_filename(data_path, filename, suffix=None):
    if suffix:
        return os.path.join(data_path, filename + '.' + suffix)
    
    return os.path.join(data_path, filename + '.csv' )

movies_metadata_fname = get_source_filename(data_path, 'movies_metadata')
ratings_fname = get_source_filename(data_path, 'ratings')
evaluation_ratings_fname = get_source_filename(data_path, 'evaluation_ratings')
test_predictions_fname = get_source_filename(data_path, 'test')
print(movies_metadata_fname)

# output_path = get_source_filename(data_path, 'predictions')
print(output_path)


/content/drive/My Drive/Colab Notebooks/Attraqt/movies_metadata.csv
/content/drive/My Drive/Colab Notebooks/Attraqt/outputs


# Read data

In [5]:
movies_metadata_df= pd.read_csv(movies_metadata_fname, ',', low_memory=False)
print(movies_metadata_df.shape)
movies_metadata_df.head()

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
ratings_df = pd.read_csv(ratings_fname, ',', low_memory=False)
print(ratings_df.shape)
ratings_df.head()

(20824175, 4)


,userId,movieId,rating,timestamp
0,1,81834,5.0,1425942133
1,1,112552,5.0,1425941336
2,1,98809,0.5,1425942640
3,1,99114,4.0,1425941667
4,1,858,5.0,1425941523


In [7]:
ratings_df['rating'].unique()

array([5. , 0.5, 4. , 2.5, 4.5, 1. , 3. , 2. , 3.5, 1.5])

# Data exploration 

In [8]:
movies_metadata_df.loc[movies_metadata_df['id'].str.contains('-')]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,False,6.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,False,7.0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,False,4.3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
movies_metadata_df['id'] = movies_metadata_df['id'].str.replace('-', '')

In [10]:
movies_metadata_df.loc[movies_metadata_df['id'].str.contains('-')]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [11]:
movies_metadata_df['id'] = movies_metadata_df['id'].astype(int)

**Drop rows in movies metadata where the votecount is nan**

In [62]:
movies_metadata_df = movies_metadata_df[movies_metadata_df['vote_count'].notna()]
movies_metadata_df.loc[movies_metadata_df['vote_count'].isna()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


# Collaborative Filtering based recommender

* The recommenders until now were interesting, but did not take into account the user ratings.
* As a result of this, the above recommenders would not give any personalized recommendation based on individual tastes.
* For this Collaborative filtering comes to our rescue. 
* Since this is a matrix factorization problem where the rows of the matrix is the same as number of unique userId and the number of columns is the same as number of unique movieId. Where as the cells of this matrix are filled with rating, whenever present else, 0. This would eventually give a sparse matrix.
* For these kind of problems **svd** comes to resuce

In [13]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,81834,5.0,1425942133
1,1,112552,5.0,1425941336
2,1,98809,0.5,1425942640
3,1,99114,4.0,1425941667
4,1,858,5.0,1425941523


In [14]:
reader = Reader()
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)
# svd = SVD(n_epochs= 10, lr_all= 0.005, reg_all= 0.4)


In [15]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [16]:
trainset = data.build_full_trainset()


# Test KNN based model


In [69]:
from surprise import KNNWithMeans

# To use item-based cosine similarity
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}}

param_grid = {"sim_options": sim_options}
knn_grid_search = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=5)
knn_grid_search.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])


In [ ]:
# get best estimator
knn = knn_grid_search.best_estimator['rmse'] 

In [ ]:
# Dump algorithm and reload it.
knn_model_file_path =  os.path.join(data_path, 'knn_trained')
# dump.dump(knn_model_file_path, algo=knn)
_, knn = dump.load(knn_model_file_path)

# Matrix factorization

In [ ]:
svd.fit(trainset)

In [17]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
svd_grid_search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)

In [18]:
svd_grid_search.fit(data)

In [19]:
results_df = pd.DataFrame.from_dict(svd_grid_search.cv_results)
results_df.head(20)

In [20]:
# best RMSE score
print(svd_grid_search.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(svd_grid_search.best_params['rmse'])

In [21]:
svd = svd_grid_search.best_estimator['rmse']
svd.fit(data.build_full_trainset())

In [22]:
from surprise import dump

# Compute predictions of the 'original' algorithm.
# test_set_for_model_eval = trainset.build_testset()
# predictions = svd.test(test_set_for_model_eval)

# Dump algorithm and reload it.
model_file_path =  os.path.join(data_path, 'svd_trained')
# dump.dump(model_file_path, algo=svd)
_, svd = dump.load(model_file_path)

# We now ensure that the algo is still the same by checking the predictions.
# predictions_loaded_algo = loaded_algo.test(test_set_for_model_eval)
# assert predictions == predictions_loaded_algo

# Submission


In [23]:
evaluation_ratings_df = pd.read_csv(evaluation_ratings_fname, ',', low_memory=False)
print(evaluation_ratings_df.shape)
evaluation_ratings_df.head()

(5198034, 2)


,userId,movieId
0,1,110
1,1,1968
2,1,4878
3,1,54503
4,1,91542


In [24]:
evaluation_ratings_df['ratings_predicitons'] = evaluation_ratings_df.apply(lambda x: svd.predict(x.userId, x.movieId), axis=1 )

In [25]:
evaluation_ratings_df.shape

(5198034, 3)

In [26]:
evaluation_ratings_df['ratings_predicitons'].head()[0]

Prediction(uid=1, iid=110, r_ui=None, est=4.18085122597756, details={'was_impossible': False})

In [27]:
evaluation_ratings_df.sort_values(['ratings_predicitons'], ascending=False)

,userId,movieId,ratings_predicitons
5198033,270896,60069,"(270896, 60069, None, 4.117408452786243, {'was..."
5198032,270896,58559,"(270896, 58559, None, 4.19720432517848, {'was_..."
5198031,270896,50872,"(270896, 50872, None, 3.99362276231029, {'was_..."
5198030,270896,48774,"(270896, 48774, None, 3.986898552005602, {'was..."
5198029,270896,48516,"(270896, 48516, None, 4.1893401886125625, {'wa..."
...,...,...,...
4,1,91542,"(1, 91542, None, 4.020480699498836, {'was_impo..."
3,1,54503,"(1, 54503, None, 3.9963045766166894, {'was_imp..."
2,1,4878,"(1, 4878, None, 4.200278164767717, {'was_impos..."
1,1,1968,"(1, 1968, None, 4.075317328521312, {'was_impos..."


In [28]:
evaluation_ratings_df.isna().sum()

userId                 0
movieId                0
ratings_predicitons    0
dtype: int64

In [29]:
evaluation_ratings_df.dtypes

userId                  int64
movieId                 int64
ratings_predicitons    object
dtype: object

In [30]:
output_file_name = get_source_filename(output_path, 'pred')
evaluation_ratings_df.to_csv(output_file_name, index=False)

In [31]:
evaluation_ratings_df['Rating'] = evaluation_ratings_df['ratings_predicitons'].apply(lambda x: x.est)
evaluation_ratings_df['was_impossible'] = evaluation_ratings_df['ratings_predicitons'].apply(lambda x: x.details['was_impossible'])

In [32]:
evaluation_ratings_df.head()

,userId,movieId,ratings_predicitons,Rating,was_impossible
0,1,110,"(1, 110, None, 4.18085122597756, {'was_impossi...",4.180851,False
1,1,1968,"(1, 1968, None, 4.075317328521312, {'was_impos...",4.075317,False
2,1,4878,"(1, 4878, None, 4.200278164767717, {'was_impos...",4.200278,False
3,1,54503,"(1, 54503, None, 3.9963045766166894, {'was_imp...",3.996305,False
4,1,91542,"(1, 91542, None, 4.020480699498836, {'was_impo...",4.020481,False


In [33]:
evaluation_ratings_df.loc[evaluation_ratings_df.was_impossible == True]

,userId,movieId,ratings_predicitons,Rating,was_impossible


In [34]:
output_file_name = get_source_filename(output_path, 'predictions_all')
evaluation_ratings_df.to_csv(output_file_name, index=False)

In [35]:
submission_df = pd.read_csv(output_file_name, ',')
submission_df.head()

,userId,movieId,ratings_predicitons,Rating,was_impossible
0,1,110,user: 1 item: 110 r_ui = None ...,4.180851,False
1,1,1968,user: 1 item: 1968 r_ui = None ...,4.075317,False
2,1,4878,user: 1 item: 4878 r_ui = None ...,4.200278,False
3,1,54503,user: 1 item: 54503 r_ui = None ...,3.996305,False
4,1,91542,user: 1 item: 91542 r_ui = None ...,4.020481,False


In [36]:
submission = submission_df[['userId', 'movieId', 'Rating']]
submission.head()

,userId,movieId,Rating
0,1,110,4.180851
1,1,1968,4.075317
2,1,4878,4.200278
3,1,54503,3.996305
4,1,91542,4.020481


In [37]:
submission.columns = ['UserId', 'MovieId', 'Rating']
submission

,UserId,MovieId,Rating
0,1,110,4.180851
1,1,1968,4.075317
2,1,4878,4.200278
3,1,54503,3.996305
4,1,91542,4.020481
...,...,...,...
5198029,270896,48516,4.189340
5198030,270896,48774,3.986899
5198031,270896,50872,3.993623
5198032,270896,58559,4.197204


In [38]:
output_file_name = get_source_filename(output_path, 'submission')
submission.to_csv(output_file_name, index=False)

# Advanced recommender

The svd recommender and content based recommender in the previous notebook work individually fine, but we can combine the results of the two techniques to build an advanced recommender.

In [ ]:
# cosine_similarity_output_filename = get_source_filename(output_path, 'cosine_similarity', '.npy')
# # read saved cosine similarity matrix
cosine_similarity = np.load(cosine_similarity_output_filename)

In [46]:
# get a data frame of movies and ids
titles = movies_metadata_df['title']
indices = pd.Series(movies_metadata_df.index, index=movies_metadata_df['title'])

In [55]:
titles.head()

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

In [53]:
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [51]:
print(indices['Jumanji'])
(titles[1])

1


'Jumanji'

In [66]:
movie_ids = movies_metadata_df[['id', 'title']]
movie_ids.head()

,id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [ ]:
def get_advanced_movies_recommendations(userId, movie_title) 
    movie_index = indices[movie_title]
    movie_id = movie_ids.loc[movie_ids['title'] == movie_title]['id']
    
    similarity_scores = list(enumerate(cosine_similarity[int(movie_index)]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:20]

    movie_indices = [i[0] for i in similarity_scores]

    movies = movies_metadata_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, x).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

**Bonus**

In [39]:
# UserId	MovieId	Rating
# userId	movieId	rating	original_title
# metrics = pd.merge(submission, user_ratings_final, left_on=["UserId", "MovieId"], right_on=["userId", "movieId"], how="inner",)

In [40]:
from itertools import product
user_ids_list = ratings_df['userId'].unique().tolist()
movies_id_list = movies_metadata_df['id'].unique().tolist()

# pairs = list(product(user_ids_list, movies_id_list))

In [41]:
print(len(user_ids_list))
print(len(movies_id_list))

270896
45430


In [42]:
print(270896 * 45436)

12308430656


In [43]:
column_names = ['UserId', 'MovieId', 'Rating']

In [44]:
count = 100
if(count % 10 == 0):
  print(count)


100


In [45]:
row_list = []
count = 0
for user_id in user_ids_list:
  for movie_id in movies_id_list:
    row_list.append([user_id, movie_id])
    if(count % 1000000 == 0):
      print(count)
    # rating = svd.predict(user_id, movie_id)
    # row_list.append({'UserId':user_id, 'MovieId':movie_id, 'Rating':rating.est })
    count = count + 1


0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000


KeyboardInterrupt: ignored

In [ ]:
len(row_list)

In [ ]:
bonus_submission_df = pd.DataFrame(row_list, columns=column_names)
bonus_submission_df.head()

**predict RMSE**

In [ ]:
from sklearn.metrics import mean_squared_error
def get_rmse(y_pred, y_test):
  rmse = mean_squared_error(y_test, y_pred, squared=False)
  print("The root mean squared error (RMSE) on test set: {:.6f}".format(rmse))
  return rmse